In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
!unzip gdrive/My\ Drive/Datasets/jena_climate_2009_2016.csv

Archive:  gdrive/My Drive/Datasets/jena_climate_2009_2016.csv.zip
  inflating: jena_climate_2009_2016.csv  
  inflating: __MACOSX/._jena_climate_2009_2016.csv  


In [4]:
fname='jena_climate_2009_2016.csv'

In [5]:
f=open(fname)

In [6]:
data=f.read()

In [7]:
lines=data.split('\n')

In [8]:
header=lines[0].split(',')
lines=lines[1:]

In [9]:
import numpy as np
mat=np.zeros((len(lines),len(header)-1))
for i,line in enumerate(lines):
  values=[float(x) for x in line.split(',')[1:]]
  mat[i,:]=values


In [10]:
mean=mat[:200000].mean(axis=0)
mat=mat-mean
std=mat[:200000].std(axis=0)
mat=mat/std


In [11]:
def generator(data, lookback, delay, min_index, max_index,shuffle=False, batch_size=128, step=6):
  if max_index is None:
    max_index = len(data) - delay - 1
  i = min_index + lookback
  while 1:
    if shuffle:
      rows = np.random.randint(min_index + lookback, max_index, size=batch_size)
    else:
      if i + batch_size >= max_index:
        i = min_index + lookback
      rows = np.arange(i, min(i + batch_size, max_index))
      i += len(rows)
    samples = np.zeros((len(rows),lookback // step,data.shape[-1]))
    targets = np.zeros((len(rows),))
    for j, row in enumerate(rows):
      indices = range(rows[j] - lookback, rows[j], step)
      samples[j] = data[indices]
      targets[j] = data[rows[j] + delay][1]
    yield samples, targets

In [12]:
lookback = 1440
step = 6
delay = 144
batch_size = 128

In [13]:
train_gen = generator(mat,lookback=lookback,delay=delay,min_index=0,max_index=200000,shuffle=True,step=step,batch_size=batch_size)
val_gen = generator(mat,lookback=lookback,delay=delay,min_index=200001,max_index=300000,step=step,batch_size=batch_size)
test_gen = generator(mat,lookback=lookback,delay=delay,min_index=300001,max_index=None,step=step,batch_size=batch_size)


In [16]:
val_steps = (300000 - 200001 - lookback)
test_steps = (len(mat) - 300001 - lookback)

1 D convnet on the csv temperature data

In [17]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop
model = Sequential()
model.add(layers.Conv1D(32, 5, activation='relu',
input_shape=(None, mat.shape[-1])))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.compile(optimizer=RMSprop(), loss='mae')
history = model.fit_generator(train_gen,
steps_per_epoch=500,
epochs=20,
validation_data=val_gen,
validation_steps=val_steps)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
499/500 [============================>.] - ETA: 0s - loss: 0.4509

KeyboardInterrupt: ignored

In [ ]:
history_dict

In [ ]:
from matplotlib import pyplot as plt
epchs=range(0,20)
plt.plot(epchs,history.history['loss'],'b',label='training loss')
plt.plot(epchs,history.history['val_loss'],'bo',label='validation loss')

Higher resolution data generators

In [ ]:
step = 3
lookback = 720
delay = 144

In [ ]:
train_gen = generator(float_data,
lookback=lookback,
delay=delay,
min_index=0,
max_index=200000,
shuffle=True,
step=step)
val_gen = generator(float_data,
lookback=lookback,
delay=delay,
min_index=200001,
max_index=300000,
step=step)
test_gen = generator(float_data,
lookback=lookback,
delay=delay,
min_index=300001,
max_index=None,
step=step)
val_steps = (300000 - 200001 - lookback) // 128
test_steps = (len(float_data) - 300001 - lookback) // 128

1 d convuolutional base with GRU

In [ ]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop
model = Sequential()
model.add(layers.Conv1D(32, 5, activation='relu',
input_shape=(None, float_data.shape[-1])))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.GRU(32, dropout=0.1, recurrent_dropout=0.5))
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer=RMSprop(), loss='mae')

In [ ]:
history = model.fit_generator(train_gen,
steps_per_epoch=500,
epochs=20,
validation_data=val_gen,
validation_steps=val_steps)

In [ ]:
from matplotlib import pyplot as plt
epchs=range(0,20)
plt.plot(epchs,history.history['loss'],'b',label='training loss')
plt.plot(epchs,history.history['val_loss'],'bo',label='validation loss')